# Resume screening
This notebook goes through the process of screening a batch of resumes against a job specification, using NLP tools. The first step looks at named-entity recognition (NER) to extract key information from the resume. Next will be a look at the cosine similarity of each resume against the job spec, with the aim of filtering out the less relevent resumes. Third will be grouping the resumes together using LDA and finally using TF-IDF to attempt to rank the resumes for the given role. 

The resumes will then be uploaded to a vector database to simplify searching for appropriate candidates in the future.

In [15]:
import pandas as pd
import spacy
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

import gensim
import gensim.corpora as corpora
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import CoherenceModel
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /Users/riz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/riz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
resume_text = pd.read_csv('../data/UpdatedResumeDataSet.csv')

### 1. NER implementation 

In [3]:
# Load pre-trained bert model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# Load pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

# NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

In [4]:
'''
skill_pattern_path = "../data/jz_skill_patterns.jsonl"
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))
'''

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [4]:
# Function to extract named entities using bert ner
def extract_entities_bert(resume_text):
    entities = ner_pipeline(resume_text)
    # Further processing to extract specific entities like name, education, etc.
    # This might require custom logic based on the structure of your resumes
    return entities

# Function to extract named entities using spaCy
def extract_entities_spacy(resume_text):
    doc = nlp(resume_text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [5]:
resume_text['Entities_spacy'] = resume_text['Resume'].apply(extract_entities_spacy)
resume_text['Entities_bert'] = resume_text['Resume'].apply(extract_entities_bert)

# Display the new DataFrame with extracted entities
print(resume_text[['Category', 'Entities_spacy', 'Entities_bert']])


         Category                                     Entities_spacy  \
0    Data Science  [(Sql, PERSON), (Java, PERSON), (JavaScript/JQ...   
1    Data Science  [(May 2013 to, DATE), (May 2017, DATE), (Expri...   
2    Data Science  [(Control System Design, ORG), (Web Developmen...   
3    Data Science  [(Tableau, GPE), (SAP HANA SQL, ORG), (SAP HAN...   
4    Data Science  [(MCA, ORG), (YMCAUST, ORG), (Faridabad, GPE),...   
..            ...                                                ...   
957       Testing  [(MS, GPE), (Basic Excel, PRODUCT), (Loyalty &...   
958       Testing  [(Team Player, ORG), (DECLARATION, PERSON), (J...   
959       Testing  [(Eagerness, NORP), (Competitive, ORG), (Janua...   
960       Testing  [(SKILLS & SOFTWARE, ORG), (MS-Power Point, OR...   
961       Testing  [(Skill Set OS, PERSON), (Windows XP/7/8/8.1/1...   

                                         Entities_bert  
0    [{'entity': 'I-MISC', 'score': 0.57808983, 'in...  
1    [{'entity': 'I-O

In [24]:
selected_rows = resume_text.iloc[:5]

# Initialize an empty list to store the transformed data
transformed_data = []

# Iterate over each row
for index, row in selected_rows.iterrows():
    # Iterate over each entity in the row
    for entity in row['Entities_bert']:
        # Append the entity data to the transformed_data list
        transformed_data.append({
            'Row': index,
            'Entity': entity['entity'],
            'Score': entity['score'],
            'Index': entity['index']
            # Add more fields here as needed
        })

# Convert the transformed data into a DataFrame
transformed_df = pd.DataFrame(transformed_data)

In [25]:
print(transformed_df)

     Row  Entity     Score  Index
0      0  I-MISC  0.578090      6
1      0   I-ORG  0.379557     33
2      0  I-MISC  0.896526     37
3      0  I-MISC  0.978586     39
4      0  I-MISC  0.774978     40
..   ...     ...       ...    ...
290    4   I-ORG  0.427012     16
291    4  I-MISC  0.964054     60
292    4  I-MISC  0.971787     73
293    4   I-ORG  0.769654    106
294    4   I-ORG  0.793049    108

[295 rows x 4 columns]


### 2. Cosine similarity
Look at cosine similarity between resume and job spec

In [13]:
# preprocess text for both specs and resumes
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in text.lower().split() if word not in stop_words]

processed_resumes = resume_text['Resume'].apply(preprocess_text)


### 3. LDA Grouping

In [11]:
# Create Dictionary and Corpus for LDA
id2word = corpora.Dictionary(processed_resumes)
corpus = [id2word.doc2bow(text) for text in processed_resumes]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, # Adjust the number of topics
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')


In [12]:
print(processed_resumes)

0      [skill, *, programming, languages:, python, (p...
1      [education, detail, may, 2013, may, 2017, b.e,...
2      [area, interest, deep, learning,, control, sys...
3      [skill, â¢, r, â¢, python, â¢, sap, hana, â...
4      [education, detail, mca, ymcaust,, faridabad,,...
                             ...                        
957    [computer, skills:, â¢, proficient, m, office...
958    [â, willingness, accept, challenges., â, p...
959    [personal, skill, â¢, quick, learner,, â¢, e...
960    [computer, skill, &, software, knowledge, ms-p...
961    [skill, set, o, window, xp/7/8/8.1/10, databas...
Name: Resume, Length: 962, dtype: object


### 4. TF-IDF for keywords

### 5. Import to vector database